In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-17 12:58:54 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-17 12:58:54 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-17 12:58:54 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-17 12:59:01,385] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-17 12:59:01 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpsruhyzyw/test.c -o /tmp/tmpsruhyzyw/test.o
2025-07-17 12:59:01 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpsruhyzyw/test.o -laio -o /tmp/tmpsruhyzyw/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-17 12:59:01 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpzh1a6otj/test.c -o /tmp/tmpzh1a6otj/test.o
2025-07-17 12:59:01 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpzh1a6otj/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpzh1a6otj/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-17 12:59:03 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-17 12:59:03 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-17 12:59:03 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-17 12:59:03 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-17 13:06:24 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-17 13:06:24 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-17 13:06:24 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-17 13:06:24 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-17 13:07:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-17 13:07:14 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [6]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-17 13:07:31 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-17 13:07:31 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-17 13:07:31 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-17 13:07:31 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-17 13:07:31 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-17 13:07:32 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-17 13:07:32 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

## Generate samples and filter by LM knowledge

In [8]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)

2025-07-17 13:07:53 src.selection.data INFO     Loaded 16 categories


In [11]:
from src.selection.data import SelectionSample, get_random_sample

sample = get_random_sample(
    people_by_category = people_by_category,
    mt = mt,
    n_distractors=5,
    get_alt_obj=False,
    # category="actor",
    obj_idx=3,
    filter_by_lm_prediction=True
)
print(sample)
print(sample.prompt)
sample.prediction

Andrey Rublev -> Caroline Garcia (3): ['Coen Brothers', 'Jane Mayer', 'Zion Williamson', 'Caroline Garcia', 'Jalen Hurts', 'Bernie Sanders']
Which person from the following list has the profession in common with Andrey Rublev?
Coen Brothers, Jane Mayer, Zion Williamson, Caroline Garcia, Jalen Hurts, Bernie Sanders.
Ans:


[PredictedToken(token=' Caroline', prob=0.66796875, logit=19.5, token_id=54491, metadata=None),
 PredictedToken(token=' None', prob=0.1689453125, logit=18.125, token_id=2290, metadata=None),
 PredictedToken(token=' And', prob=0.06201171875, logit=17.125, token_id=1628, metadata=None),
 PredictedToken(token=' Bernie', prob=0.017822265625, logit=15.875, token_id=30324, metadata=None),
 PredictedToken(token=' J', prob=0.0157470703125, logit=15.75, token_id=622, metadata=None)]

In [ ]:
##########################################################
save_dir = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, "selection", mt.name.split("/")[-1], "profession"
)
os.makedirs(save_dir, exist_ok=True)
file_name = "filtered_samples.json"

LIMIT = 10000
N_DISTRACTORS = 5
##########################################################

from src.utils.experiment_utils import set_seed
set_seed(123456)

filtered_samples = []
while len(filtered_samples) < LIMIT:
    sample = get_random_sample(
        people_by_category=people_by_category,
        mt=mt,
        n_distractors=N_DISTRACTORS,
        get_alt_obj=False,
        filter_by_lm_prediction=True
    )
    sample.detensorize()
    filtered_samples.append(sample)
    print(f"Collected {len(filtered_samples)}/{LIMIT} samples. {len(filtered_samples) / LIMIT * 100:.2f}%")

with open(os.path.join(save_dir, file_name), "w") as f:
    json.dump([s.to_dict() for s in filtered_samples], f, indent=2)

2025-07-17 13:41:44 src.utils.experiment_utils INFO     setting all seeds to 123456


Collected 1/1000 samples. 0.10%
Collected 2/1000 samples. 0.20%
2025-07-17 13:41:46 src.selection.data ERROR    Sample = Oprah Winfrey -> Warren Buffett (4): ['Tyreek Hill', 'Wolf Blitzer', 'Ralph Lauren', 'Phil Mickelson', 'Warren Buffett', 'James Patterson']
Top prediction " Wolf"[26296] (p=0.539, logit=19.000) does not match the object Warren Buffett[26713, " Warren"].
Retrying ...

Collected 3/1000 samples. 0.30%
Collected 4/1000 samples. 0.40%
Collected 5/1000 samples. 0.50%
2025-07-17 13:41:50 src.selection.data ERROR    Sample = Billie Eilish -> Harry Styles (0): ['Harry Styles', 'Virgil Abloh', 'Haruki Murakami', 'James Cameron', 'Trevor Noah', 'Errol Spence Jr.']
Top prediction " None"[2290] (p=0.547, logit=18.250) does not match the object Harry Styles[14253, " Harry"].
Retrying ...

Collected 6/1000 samples. 0.60%
Collected 7/1000 samples. 0.70%
Collected 8/1000 samples. 0.80%
Collected 9/1000 samples. 0.90%
Collected 10/1000 samples. 1.00%
Collected 11/1000 samples. 1.10%
C

In [69]:
with open(os.path.join(save_dir, "filtered_samples.json"), "r") as f:
    loaded_samples = json.load(f)
loaded_samples = [SelectionSample.from_dict(s) for s in loaded_samples]

In [70]:
print(loaded_samples[0].prompt)

Which person from the following list has the profession in common with Canelo Álvarez?
George Kambosos Jr., Morgan Freeman, Rihanna, Bryson DeChambeau, Diane von Furstenberg, Elizabeth Warren.
Ans:


## Cache last token states for the generated samples

In [71]:
with open(os.path.join(save_dir, "filtered_samples.json"), "r") as f:
    filtered_samples = json.load(f)
filtered_samples = [SelectionSample.from_dict(s) for s in filtered_samples]

len(filtered_samples)

1000

In [72]:
#######################################################################################
cache_dir = os.path.join(save_dir, "cached_states")

all_layers = (
    [mt.embedder_name]  # embeddings
    + mt.layer_names  # residual
    + [mt.mlp_module_name_format.format(i) for i in range(mt.n_layer)]  # mlp outputs
    + [mt.attn_module_name_format.format(i) for i in range(mt.n_layer)]  # attn outputs
)
TOKEN_POSITION = -1

locations = [(layer_name, TOKEN_POSITION) for layer_name in all_layers]
#######################################################################################
os.makedirs(cache_dir, exist_ok=True)

import numpy as np
from src.utils.typing import TokenizerOutput
from src.functional import get_hs, detensorize
from src.tokens import prepare_input

for idx, sample in enumerate(filtered_samples):
    # inputs = TokenizerOutput(data = sample.metadata["tokenized"]).to(mt.device)
    inputs = prepare_input(prompts=sample.prompt, tokenizer=mt)
    sample.detensorize()

    cache = {"sample": sample.to_dict(), "states": {}}
    states = get_hs(
        mt=mt,
        input=inputs,
        locations=locations,
        return_dict=True,
    )

    for (layer_name, tok_idx), state in states.items():
        cache["states"][layer_name] = state.detach().to(torch.float32).cpu().numpy()

    cache = detensorize(cache)
    np.savez_compressed(
        os.path.join(cache_dir, f"sample_{idx}.npz"), **cache, allow_pickle=True
    )

    logger.info(
        f"Processed sample {idx + 1}/{len(filtered_samples)} ({(idx + 1) / len(filtered_samples) * 100:.2f}%)"
    )

2025-07-17 15:04:49 __main__ INFO     Processed sample 1/1000 (0.10%)
2025-07-17 15:04:55 __main__ INFO     Processed sample 2/1000 (0.20%)
2025-07-17 15:05:01 __main__ INFO     Processed sample 3/1000 (0.30%)
2025-07-17 15:05:06 __main__ INFO     Processed sample 4/1000 (0.40%)
2025-07-17 15:05:11 __main__ INFO     Processed sample 5/1000 (0.50%)
2025-07-17 15:05:17 __main__ INFO     Processed sample 6/1000 (0.60%)
2025-07-17 15:05:22 __main__ INFO     Processed sample 7/1000 (0.70%)
2025-07-17 15:05:28 __main__ INFO     Processed sample 8/1000 (0.80%)
2025-07-17 15:05:33 __main__ INFO     Processed sample 9/1000 (0.90%)
2025-07-17 15:05:38 __main__ INFO     Processed sample 10/1000 (1.00%)
2025-07-17 15:05:44 __main__ INFO     Processed sample 11/1000 (1.10%)
2025-07-17 15:05:49 __main__ INFO     Processed sample 12/1000 (1.20%)
2025-07-17 15:05:54 __main__ INFO     Processed sample 13/1000 (1.30%)
2025-07-17 15:06:00 __main__ INFO     Processed sample 14/1000 (1.40%)
2025-07-17 15:0

### Testing by loading the cached states

In [52]:
cache_dir = os.path.join(save_dir, "cached_states")
os.listdir(cache_dir)

['sample_7.npz',
 'sample_2.npz',
 'sample_6.npz',
 'sample_4.npz',
 'sample_9.npz',
 'sample_1.npz',
 'sample_8.npz',
 'sample_0.npz',
 'sample_5.npz',
 'sample_3.npz']

In [55]:
sample_states = np.load(os.path.join(cache_dir, "sample_0.npz"), allow_pickle=True)
sample_states.files

['sample', 'states']

In [60]:
sample = SelectionSample.from_dict(sample_states["sample"].item())
print(sample.prompt)
print(sample.prediction)

Which person from the following list has the profession in common with Wesley Lowery?
Quentin Tarantino, Chris Hemsworth, Hideki Matsuyama, Yamiche Alcindor, Alessandro Michele, Justin Jefferson.
Ans:
(PredictedToken(token=' Y', prob=0.65234375, logit=18.75, token_id=816, metadata=None), PredictedToken(token=' Wesley', prob=0.060791015625, logit=16.375, token_id=58706, metadata=None), PredictedToken(token=' The', prob=0.060791015625, logit=16.375, token_id=578, metadata=None), PredictedToken(token=' None', prob=0.041748046875, logit=16.0, token_id=2290, metadata=None), PredictedToken(token=' Journal', prob=0.02099609375, logit=15.3125, token_id=10139, metadata=None))


In [66]:
states = {
    layer_name: torch.Tensor(value).to(mt.device)
    for layer_name, value in sample_states["states"].item().items()
}

states

{'model.embed_tokens': tensor([ 3.1233e-05,  2.3651e-03,  2.6855e-03,  ...,  8.6594e-04,
         -2.4567e-03,  3.0975e-03], device='cuda:0'),
 'model.layers.0': tensor([-0.0006,  0.0035,  0.0023,  ..., -0.0010, -0.0036, -0.0028],
        device='cuda:0'),
 'model.layers.1': tensor([-0.0044,  0.0014,  0.0029,  ...,  0.0005, -0.0060, -0.0008],
        device='cuda:0'),
 'model.layers.2': tensor([-0.0126, -0.0017, -0.0043,  ..., -0.0024,  0.0028, -0.0116],
        device='cuda:0'),
 'model.layers.3': tensor([-0.0204,  0.0040,  0.0055,  ..., -0.0060,  0.0059, -0.0034],
        device='cuda:0'),
 'model.layers.4': tensor([-0.0208, -0.0048,  0.0054,  ..., -0.0120,  0.0119, -0.0052],
        device='cuda:0'),
 'model.layers.5': tensor([-0.0189, -0.0087,  0.0023,  ..., -0.0012,  0.0225, -0.0078],
        device='cuda:0'),
 'model.layers.6': tensor([-0.0082,  0.0036,  0.0079,  ..., -0.0079,  0.0325, -0.0173],
        device='cuda:0'),
 'model.layers.7': tensor([-0.0195,  0.0018,  0.0222,  ...,